## Models and Maps with Ipyleaflet

In this next section, we're going to use all of what you have learned so far to create a heatwave forecasting map which lets users explore future events. 

Please note, the model for air temperature in this section is basically totally made up. It makes some intuitive sense but the parameters are basically random, and probably will not give any reasonable estimate, it is purely to demonstrate how to use a model to create data which will go onto a map. 

In [ ]:
import numpy as np

# First, lets define a really simple model for the air temperature of point on the land surface. We will model this as a linear combination of 
# solar irradiance, NDVI, NDBI, and NDWI. We will also add some random noise to the model to make it more realistic. 
# This relies on the intuition of the urban heat island effect, where the emissivity and other characteristics of the land surface
# affect temperature. 

def model_airtemp(solar_irradiance, ndvi, ndbi, ndwi, c=25, ndvi_beta=-3, ndbi_beta=4, ndwi_beta=-2):
    airtemp =  ndvi_beta*ndvi + ndbi_beta*ndbi + ndwi_beta*ndwi + np.random.normal(-1, 1) + c + solar_irradiance
    return airtemp

# Fun question - why do I add the solar_arradiance to the model ad the end?

In [ ]:
import ipywidgets as widgets
from ipyleaflet import Map, ImageOverlay
import rasterio as rio

m = Map(center=(40.7128, -74.0060), zoom=12)

# First, we will load the spectral indices data from the raster files. 

ndvi_array = rio.open('data/ndvi.tif').read(1)
ndwi_array = rio.open('data/ndwi.tif').read(1)
ndbi_array = rio.open('data/ndbi.tif').read(1)

# All three of these arrays have the same shape, so we can use the shape and extent of one of them to define the extent of air temperature data.

image_extent = rio.open('data/ndvi.tif').bounds
SW = (bounds.left, bounds.bottom)
NE = (bounds.right, bounds.top)
bounds_tuple = (SW, NE)

# Now, lets create the airtemperature array which holds the data
airtemp_array = model_airtemp(3, ndvi_array, ndbi_array, ndwi_array)

from PIL import Image

array = np.moveaxis(airtemp_array, 0, -1)

array_max = np.max(array)
array_min = np.min(array)
array = np.clip((array - array_min) / (array_max - array_min) * 255, 0, 255)
array = array.as_type(np.uint8)

image = Image.fromarray(array)


jpeg_path_temp = SAVE_FILE_FUNCTION(PTH)

# Now we can add the image to the map
image_layer = ImageOverlay(url=jpeg_path_temp, bounds=bounds_tuple)
m.add_layer(image_layer)
display(m)

Now that we have added the data to the map, its time to put it all together! 

Using what you have learned in the previous notebooks, your job is to:
1. Create a slider to alter solar_irradiance between 10 and 23
2. Use an observer to calculate a new air temperature raster each time the slider is changed
3. Clear the map of its old layers, and add the new raster once it is calculated
4. Create an additional panel for tuning the coefficients of the model: ndvi_beta, ndwi_beta, and ndbi_beta between -5 and 5
5. Use containers to compose these widgets together so the map is on one side, and controls are on the other

To get started, I will provide a function which automatically converts the air temperature array to a jpeg file


In [ ]:
def array_to_jpeg(array, jpeg_path='temp.jpeg'):
    array = np.moveaxis(airtemp_array, 0, -1)

    array_max = np.max(array)
    array_min = np.min(array)
    array = np.clip((array - array_min) / (array_max - array_min) * 255, 0, 255)
    array = array.as_type(np.uint8)

    image = Image.fromarray(array)

    jpeg_path_temp = SAVE_FILE_FUNCTION(PTH)
    return jpeg_path



In [ ]:
## Bonus Exercise! 
# The image added to the map is currently gray-scale, and doesn't contain any information about the scale of the values. 
# Your task is to update the widgets you have created above to generate colour images using a colour map of your choice. 
# We have to manually convert the single-band array to a 3-band colour image.
# We can do that using the following function
import matplotlib
import matplotlib.cm as cm

def greyscale_to_colour(greyscale_array):
    array_max = np.max(array)
    array_min = np.min(array)
    # Matplotlib wants the data in range [0, 1], so we need to normalise the data
    normaliser = matplotlib.colors.Normalize(vmin=array_min, vmax=array_max, clip=True)
    scalar_mapper = cm.ScalarMappable(norm=normaliser, cmap='magma')
    colour_array = scalar_mapper.to_rgba(greyscale_array)
    return colour_array[:,:,:-1] #drop the alpha channel

#The rest is up to you